In [65]:
MODEL = "gpt-4o-2024-08-06"
#MODEL = "gpt-4o-mini"
#MODEL = "gpt-3.5-turbo"
#content = ""
#prompt = [{"role": "user", "content": content}]
loops = 1
file = "chat/3"

import os
from openai import OpenAI
import pandas as pd
from IPython.display import display_markdown
import copy

client = OpenAI()
df_path = os.path.join(os.getcwd(),"completions")
chat = True


def transform_roles(chat,assistant_transform_type = "user-assistant"):
    prompt_messages = copy.deepcopy(chat)
    if assistant_transform_type == "user-assistant":
      i = 1
      while i <= len(prompt_messages) and prompt_messages[-i]["role"] != "system":
        if i%2 == 1:
          prompt_messages[-i]["role"] = "user"
        else:
          prompt_messages[-i]["role"] = "assistant"
        i += 1
    else:
      for prompt_message in prompt_messages:
        if prompt_message["role"] == "assistant":
          prompt_message["role"] = assistant_transform_type
    return prompt_messages

In [66]:
# Load completions from json, if continuing with a file generated in the past

try:
    with open(os.path.join(df_path,file), "r") as f:
        completions = pd.read_json(f, lines=True)

# If not, create a new completions database

except FileNotFoundError:
    completions = pd.DataFrame([],columns=['Response', 'Eval', 'Data', 'Instruction'])

In [67]:
for i in range(loops):
    completion = client.chat.completions.create(
        messages = prompt,
        model = MODEL,
        )
    response = completion.choices[0].message.content

    data = {'Response': response, 'Eval': "", 'Data': {}, 'Instruction': prompt}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = {"Completion":completion.to_dict()}
    completions = pd.concat([completions, data],ignore_index=True)
    completions.to_json(os.path.join(df_path, file), lines=True, orient="records")

In [44]:
completions

,Response,Eval,Data,Instruction
0,Hello! I'm here to help with any questions or ...,,{'Completion': {'id': 'chatcmpl-9wUKE1esX2JTyi...,"[{'role': 'assistant', 'content': ''}, {'role'..."
1,Hello! I'm here to assist you with any questio...,,{'Completion': {'id': 'chatcmpl-9wUKF1U6FjiZyx...,"[{'role': 'assistant', 'content': ''}, {'role'..."
2,"Hi there! I'm here to help, so feel free to as...",,{'Completion': {'id': 'chatcmpl-9wUKGGpnWXIWRi...,"[{'role': 'assistant', 'content': ''}, {'role'..."
3,Hello! I'm here to help with any questions you...,,{'Completion': {'id': 'chatcmpl-9wUKHzgvsOFFYw...,"[{'role': 'assistant', 'content': ''}, {'role'..."
4,Hello! I'm here to help with any questions or ...,,{'Completion': {'id': 'chatcmpl-9wUKIfboTlr6of...,"[{'role': 'assistant', 'content': ''}, {'role'..."
...,...,...,...,...
95,Hello! I'm here to help with any questions or ...,,{'Completion': {'id': 'chatcmpl-9wULtxX0Tug6Qi...,"[{'role': 'assistant', 'content': ''}, {'role'..."
96,Hello! I'm here to help with any questions you...,,{'Completion': {'id': 'chatcmpl-9wULuIdicKjp3Q...,"[{'role': 'assistant', 'content': ''}, {'role'..."
97,Hi! I'm here to help with any questions you ha...,,{'Completion': {'id': 'chatcmpl-9wULyQNvqIpvci...,"[{'role': 'assistant', 'content': ''}, {'role'..."
98,Hi there! I'm here to help with any questions ...,,{'Completion': {'id': 'chatcmpl-9wULyTqU3AHZ4S...,"[{'role': 'assistant', 'content': ''}, {'role'..."


In [11]:
completions.to_json(os.path.join(df_path, file), lines=True, orient="records")

In [60]:
n = 41
completions.loc[n,"Eval"] = "Best, probably. Longest."

In [61]:
completions.iloc[41]

Response       Hello! I'm here to help with any questions you...
Eval                                    Best, probably. Longest.
Data           {'Completion': {'id': 'chatcmpl-9wUL5cE1OPtosa...
Instruction    [{'role': 'assistant', 'content': ''}, {'role'...
Name: 41, dtype: object

In [24]:
completions

,Response,Eval,Data,Instruction
0,Hello! How can I assist you today?,Best,{'Completion': {'id': 'chatcmpl-9wPhCemTLIU9rK...,"[{'role': 'user', 'content': ''}, [{'role': 'a..."
1,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPhDUWFpckVaq...,"[{'role': 'user', 'content': ''}]"
2,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPhDwjYX970LS...,"[{'role': 'user', 'content': ''}]"
3,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPhEM6HEaNeZW...,"[{'role': 'user', 'content': ''}]"
4,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPhFU3uvCpw0i...,"[{'role': 'user', 'content': ''}]"
...,...,...,...,...
95,It seems you've submitted an empty message. Ho...,,{'Completion': {'id': 'chatcmpl-9wPiVTdzVMfOAG...,"[{'role': 'user', 'content': ''}]"
96,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPiWXaGj5uqvW...,"[{'role': 'user', 'content': ''}]"
97,It seems like your message got cut off or you ...,,{'Completion': {'id': 'chatcmpl-9wPiWzXUC8Zwer...,"[{'role': 'user', 'content': ''}]"
98,Hello! How can I assist you today?,,{'Completion': {'id': 'chatcmpl-9wPiXkuFd5J53C...,"[{'role': 'user', 'content': ''}]"


In [23]:
completions[completions["Eval"].apply(lambda x: x.startswith("Best"))].iloc[0]["Instruction"]

[{'role': 'user', 'content': ''},
 [{'role': 'assistant', 'content': 'Hello! How can I assist you today?'}],
 [{'role': 'assistant', 'content': 'Hello! How can I assist you today?'}],
 [{'role': 'assistant', 'content': 'Hello! How can I assist you today?'}],
 [{'role': 'assistant', 'content': 'Hello! How can I assist you today?'}],
 {'role': 'assistant', 'content': 'Hello! How can I assist you today?'},
 {'role': 'assistant', 'content': 'Hello! How can I assist you today?'},
 {'role': 'assistant', 'content': 'Hello! How can I assist you today?'},
 {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}]

In [62]:
best_response = completions[completions["Eval"].apply(lambda x: x.startswith("Best"))].iloc[0]["Response"]
if chat:
    messages = copy.deepcopy(completions[completions["Eval"].apply(lambda x: x.startswith("Best"))].iloc[0]["Instruction"])
    messages.append({"role": "assistant", "content": best_response})
    prompt = transform_roles(messages)
else:
    prompt = [{"role": "user", "content": content}]

[{'role': 'assistant', 'content': ''}, {'role': 'user', 'content': 'Hello! How can I assist you today?'}, {'role': 'assistant', 'content': "Hello! I'm here to help with any questions you have or information you need. Feel free to ask about a wide range of topics, including technology, science, general knowledge, or specific inquiries you might have. How can I assist you today?"}]


In [63]:
prompt

[{'role': 'user', 'content': ''},
 {'role': 'assistant', 'content': 'Hello! How can I assist you today?'},
 {'role': 'user',
  'content': "Hello! I'm here to help with any questions you have or information you need. Feel free to ask about a wide range of topics, including technology, science, general knowledge, or specific inquiries you might have. How can I assist you today?"}]

In [74]:
content

'Hello! How can I assist you today?'

In [ ]:
completions.sort_values("Response")

In [75]:
completions.sort_values("Response")["Response"].to_list()

['Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist you today?',
 'Hello! How can I assist